### 朴素贝叶斯算法流程：

朴素贝叶斯是经典的生成学习方法。即：由训练数据学习联合概率分布P（X，Y），然后求得后验概率分布P（Y｜X）。
也就是利用训练数据学习P（X｜Y）和P（Y）的估计，得到联合概率分布：P（X，Y）= P（Y）P（X｜Y）。
概率估计方法可以是极大似然估计，也可以是贝叶斯估计。

1、计算先验概率P（Y=ck）和条件概率P（X=x｜Y=ck）；  
2、对于给定的实例X，计算：P（Y=ck）乘以连乘【P（X=xi｜Y=ck）】，X的各个分类特征之间相互独立，即条件独立性假设；  
3、确定实例X的嘞类：y=P(Y=ck|X=x) = argmaxP（Y=ck）乘以连乘【P（X=xi｜Y=ck）】。  

下面我们给出一个垃圾邮件分类的例子。
参考了https://github.com/Asia-Lee/Naive_Bayes/blob/master/Email_NB.py

In [88]:
#coding:utf-8
import os, sys
import numpy as np;
import random
import re

In [89]:
"""
函数说明:接收一个大字符串并将其解析为字符串列表
"""
def textParse(bigString):  # 将字符串转换为字符列表
    listOfTokens = re.split(r'\W+', bigString)  # 将特殊符号作为切分标志进行字符串切分，即非字母、非数字
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]  # 除了单个字母，例如大写的I，其它单词变成小写

"""
函数说明:将切分的实验样本词条整理成不重复的词条列表，也就是词汇表
Parameters:
    dataSet - 整理的样本数据集
Returns:
    vocabSet - 返回不重复的词条列表，也就是词汇表
"""
def createVocabList(dataSet):
    vocabSet = set([])  # 创建一个空的不重复列表
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # 取并集
    return list(vocabSet)

"""
函数说明:根据vocabList词汇表，将inputSet向量化，向量的每个元素为1或0
Parameters:
    vocabList - createVocabList返回的列表
    inputSet - 切分的词条列表
Returns:
    returnVec - 文档向量,词集模型
"""
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)               #创建一个其中所含元素都为0的向量
    for word in inputSet:                          #遍历每个词条
        if word in vocabList:                      #如果词条存在于词汇表中，则置1
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec        #返回文档向量





In [90]:
def navie_bayes(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)  # 计算训练的文档数目
    numWords = len(trainMatrix[0])  # 计算每篇文档的词条数
    
    # 1、计算先验概率P（Y=0）属于非垃圾邮件，和P（Y=1）属于垃圾邮件,# 文档属于垃圾邮件类的概率
    pAbusive = sum(trainCategory) / float(numTrainDocs)  #trainCategory里面
                    #垃圾邮件为1，非垃圾邮件为0，sum(trainCategory)代表所有垃圾邮件的个数，那么这个地方求得的就是
                    #垃圾邮件的先验概率
    
    pNoAbusive = 1- pAbusive
    
    
    # 2、用贝叶斯估计计算条件概率， 这里为了防止所估计的概率值为0，我们让lambda=1，即采用拉普拉斯平滑
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)  # 创建numpy.ones数组,词条出现数初始化为1,拉普拉斯平滑
    p0Denom = 1.0
    p1Denom = 1.0  # 分母初始化为1 ,拉普拉斯平滑
    for i in range(numTrainDocs):
        # 2.1、计算垃圾邮件的条件概率P（X=x｜Y=1）
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:  # 统计属于非垃圾类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = (p1Num / p1Denom)
    p0Vect = (p0Num / p0Denom)   #取对数，防止下溢出

    return p0Vect, p1Vect, pAbusive  # 返回属于正常邮件类的条件概率数组，属于垃圾邮件类的条件概率数组，文档属于垃圾邮件类的概率
    

def inference(vec2Classify, p0Vec, p1Vec, pClass1):
    # 3、确定实例X的嘞类：y=P(Y=ck|X=x) = argmaxP（Y=ck）乘以连乘【P（X=xi｜Y=ck）】。
    p1=sum(vec2Classify*p1Vec)*(pClass1)
    p0=sum(vec2Classify*p0Vec)*(1.0-pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [91]:
def main():
    dataPath = './data'
    docList = []
    classList = []
    fullText = []
    for i in range(1, 26):  # 遍历25个txt文件
        wordList = textParse(open('./data/spam/%d.txt' % i, 'r', encoding='ISO-8859-1').read())  # 读取每个垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(1)  # 标记垃圾邮件，1表示垃圾文件
        wordList = textParse(open('./data/ham/%d.txt' % i, 'r', encoding='ISO-8859-1').read())  # 读取每个非垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(0)  # 标记正常邮件，0表示正常文件
    vocabList = createVocabList(docList)  # 创建词汇表，不重复
    trainingSet = list(range(50))
    testSet = []  # 创建存储训练集的索引值的列表和测试集的索引值的列表
    for i in range(10):  # 从50个邮件中，随机挑选出40个作为训练集,10个做测试集
        randIndex = int(random.uniform(0, len(trainingSet)))  # 随机选取索索引值
        testSet.append(trainingSet[randIndex])  # 添加测试集的索引值
        del (trainingSet[randIndex])  # 在训练集列表中删除添加到测试集的索引值
    trainMat = []
    trainClasses = []  # 创建训练集矩阵和训练集类别标签系向量
    for docIndex in trainingSet:  # 遍历训练集
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))  # 将生成的词集模型添加到训练矩阵中
        trainClasses.append(classList[docIndex])  # 将类别添加到训练集类别标签系向量中
    
    p0V, p1V, pSpam = navie_bayes(np.array(trainMat), np.array(trainClasses))  # 训练朴素贝叶斯模型

        
    errorCount = 0  # 错误分类计数
    for docIndex in testSet:  # 遍历测试集
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])  # 测试集的词集模型
        if inference(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:  # 如果分类错误
            errorCount += 1  # 错误计数加1
            print("分类错误的测试集：", docList[docIndex])
    print('错误率：%.2f%%' % (float(errorCount) / len(testSet) * 100))

In [92]:
if __name__ == '__main__':
    main()

错误率：0.00%
